## Random ray direction and ray origins

### Imports

In [107]:
import numpy as np
import torch
import trimesh
from enum import Enum


torch_device = 'cpu'


### Generating rays from spherical coordinates

In [2]:
# generate random spehrical coordinates
def generate_random_sperical_coordinates():
    # Step 1: Radius (r)
    radius = np.random.uniform(1, 3)  # You can adjust the range as needed
    
    # Step 2: Polar Angle (θ)
    polar_angle = np.random.uniform(0, np.pi)  # Range: [0, π]

    # Step 3: Azimuthal Angle (φ)
    azimuthal_angle = np.random.uniform(0, 2 * np.pi)  # Range: [0, 2π]

    return radius, polar_angle, azimuthal_angle

def spherical_to_cartesian(radius, polar_angle, azimuthal_angle):
    x = radius * np.sin(polar_angle) * np.cos(azimuthal_angle)
    y = radius * np.sin(polar_angle) * np.sin(azimuthal_angle)
    z = radius * np.cos(polar_angle)
    
    return x, y, z

def compute_ray_from_spherical_coordinates():
    # Convert spherical to Cartesian coordinates
    radius, polar_angle, azimuthal_angle = generate_random_sperical_coordinates()
    x, y, z = spherical_to_cartesian(radius, polar_angle, azimuthal_angle)

    # Ray origin is the Cartesian coordinates
    ray_origin = (x, y, z)

    # Ray direction is a normalized vector in the same direction
    ray_length = np.sqrt(x**2 + y**2 + z**2)
    ray_direction = (x/ray_length, y/ray_length, z/ray_length)

    return ray_origin, ray_direction

We will use following algorithm to generate different camera poses:
* generate uniform random $\theta \in [0, \pi]$ and $\phi \in [0, 2\pi]$
* convert spherical coordinates into Cartesian coordinates using transformation formula <br>
$x = R\sin(\theta)\cos(\phi)$ <br>
$y = R\sin(\theta)\sin(\phi)$ <br>
$z = R\cos(\theta)$ <br>
This is going to be a camera position
* convert Cartesian coordinates into camera position coordinates, thus creating camera2world matrix with
$up = {0, 1, 0}$ always assuming that that camera is moving upwards, aligned with y axis <br>
$forward = cameraPosition$ <br>
$right =  forward*up$  <br>
$up = forward * right$ <br>

* generate ray directions, assuming camera is placed at origin
* adjust ray directions using camera poses
After that we would need bound our camera position to the concrete direction - front, rear, up etc for the right prompt generation. For each position it would be easy to do if we know spherical coordinates.

We would use previsouly generated spherical coordinates to establish thresholds for each concrete position



In [160]:
class Directions(Enum):
    FRONT = 0,      np.array([255, 0, 0, 255])   # phi \in [-front/2, front/2)
    SIDE_LEFT = 1,  np.array([0, 255, 0, 255])   # phi \in [front/2, 180-front/2)
    BACK = 2,       np.array([0, 0, 255, 255])   # phi \in [180-front/2, 180+front/2)
    SIDE_RIGHT = 3, np.array([255, 255, 0, 255]) # phi \in [180+front/2, 360-front/2)
    TOP = 4,        np.array([255, 0, 255, 255]) # theta \in [0, overhead]
    BOTTOM = 5,     np.array([0, 255, 255, 255]) # theta \in [180-overhead, 180]
    def __int__(self):
        return self.value[0]
    
    def get_color(self):
        return self.value[1]

def normalize(x):
    return x / torch.sqrt(torch.sum(x**2, keepdim=True, dim=-1))


# taken from https://github.com/ashawkey/stable-dreamfusion/blob/main/nerf/provider.py
def get_directions_from_spherical(thetas, phis, overhead_angle, front_angle):
    dirs = np.empty(thetas.shape[0], dtype=Directions)
    # first determine by phis
    #phis = phis % (2 * np.pi)
    dirs[(phis < front_angle / 2) | (phis >= 2 * np.pi - front_angle / 2)] = Directions.FRONT
    dirs[(phis >= front_angle / 2) & (phis < np.pi - front_angle / 2)] = Directions.SIDE_LEFT
    dirs[(phis >= np.pi - front_angle / 2) & (phis < np.pi + front_angle / 2)] = Directions.BACK
    dirs[(phis >= np.pi + front_angle / 2) & (phis < 2 * np.pi - front_angle / 2)] = Directions.SIDE_RIGHT
    # override by thetas
    dirs[thetas <= overhead_angle] = Directions.TOP
    dirs[thetas >= (np.pi - overhead_angle)] = Directions.BOTTOM
    return dirs


def generate_random_poses(size, theta_range = np.array([0, 180]),
                          phi_range = np.array([0, 360]),
                          radius_range = np.array([1, 1.5]),
                          front_angle = 60,
                          overhead_angle = 30):
    # convert theta range and phi range to radians
    # as sin, cos function expect radians
    theta_range = theta_range * (np.pi / 180)
    phi_range = phi_range * (np.pi / 180)
    front_angle = front_angle * (np.pi / 180)
    overhead_angle = overhead_angle * (np.pi / 180)

    # generate spherical coordinates
    thetas = torch.rand(size) * (theta_range[1] - theta_range[0]) + theta_range[0]
    phis =  torch.rand(size) * (phi_range[1] - phi_range[0]) + phi_range[0]
    radius = torch.rand(size) * (radius_range[1] - radius_range[0]) + radius_range[0]
    #phis[phis < 0] += 2 * np.pi
    # transform spehrical coordinates to the point on the sphere in cartesian coordinates
    x = radius * torch.sin(thetas) * torch.cos(phis)
    y = radius * torch.sin(thetas) * torch.sin(phis)
    z = radius * torch.cos(thetas)

    camera_position = torch.stack((x, y, z), dim=-1)

    # transform cartesian coordinates to the camera coordinates
    # initial camera UP trajectory
    up = torch.Tensor([0, 1, 0]).repeat(size, 1)

    forward = camera_position
    right = normalize(torch.cross(forward, up, dim=-1))
    up = normalize(torch.cross(forward, right, dim=-1))

    # create translation matrix
    poses = torch.empty((3, 4), dtype=torch.float).repeat(size, 1, 1)
    poses[:, :3, :3] = torch.stack((right, up, forward), dim=-1)
    poses[:, :3, 3] = camera_position

    directions = get_directions_from_spherical(thetas, phis, overhead_angle, front_angle)

    return poses, directions

To test what we've generated let's visualise resulted poses

In [181]:
# taken from https://github.com/ashawkey/stable-dreamfusion/blob/main/nerf/provider.py
def visualize_poses(poses, dirs, size=0.1):
    # poses: [B, 4, 4], dirs: [B]
    #print(poses.shape)
    axes = trimesh.creation.axis(axis_length=4)
    sphere = trimesh.creation.icosphere(radius=1)
    objects = [axes, sphere]

    for pose, dir in zip(poses, dirs):
        # a camera is visualized with 8 line segments.
        pos = pose[:3, 3]
        a = pos + size * pose[:3, 0] + size * pose[:3, 1] - size * pose[:3, 2]
        b = pos - size * pose[:3, 0] + size * pose[:3, 1] - size * pose[:3, 2]
        c = pos - size * pose[:3, 0] - size * pose[:3, 1] - size * pose[:3, 2]
        d = pos + size * pose[:3, 0] - size * pose[:3, 1] - size * pose[:3, 2]

        segs = np.array([[pos.numpy(), a.numpy()],
                         [pos.numpy(), b.numpy()],
                         [pos.numpy(), c.numpy()],
                         [pos.numpy(), d.numpy()],
                         [a.numpy(), b.numpy()],
                         [b.numpy(), c.numpy()],
                         [c.numpy(), d.numpy()],
                         [d.numpy(), a.numpy()]])
        
        #print(segs)
        segs = trimesh.load_path(segs)

        # different color for different dirs
        segs.colors = np.array([dir.get_color()]).repeat(len(segs.entities), 0)

        objects.append(segs)

    trimesh.Scene(objects).show(viewer='gl')

In [182]:
poses, dirs = generate_random_poses(100)

visualize_poses(poses, dirs)


In [154]:
import sys
sys.path.append("..") 
from tiny_nerf.utils.ray_utils import get_ray_directions, get_rays

default_fov = 20
H = 10
W = 10
focal = H / (2 * np.tan(np.deg2rad(default_fov) / 2))

directions = get_ray_directions(H, W, focal)
poses, dirs = generate_random_poses(10)

ray_origin, ray_direction = get_rays(directions, poses[0])

In [155]:
def visualize_rays(origins, directions, size = 0.1):
    axes = trimesh.creation.axis(axis_length=4)
    objects = [axes]

    for origin, direction in zip(origins, directions):
        end = origin + direction
        segs = np.array([origin.numpy(), end.numpy()], dtype='object')
        segs = trimesh.load_path(segs)
        objects.append(segs)

    trimesh.Scene(objects).show(viewer='gl')

In [156]:
visualize_rays(ray_origin, ray_direction)